In [1]:
from __future__ import division, print_function, absolute_import
import tensorflow as tf
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

/Users/Supra/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [14]:
# Training Parameters
learning_rate = 0.001
num_steps = 500
batch_size = 128
display_step = 10

# Network Parameters
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [12]:
test_v = tf.Variable(tf.random_normal([5, 5, 1, 32]))
# get_shape() returns an array with dimensions of the tensor and as_list returns it as a list
v = test_v.get_shape().as_list()[0]
print(v)

5


In [25]:
# wrappers for simplicity
def conv2d(x, W, b, strides):
    # conv2d wrapper, with bias and relu activation NHWC format
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k):
    # max_pool wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

# model creation
def conv_net_2d(x, weights, biases, dropout):
    # -1 means whatever be the batchsize, 28*28 is the image size, 1 is the channel
    x = tf.reshape(x, [-1,28,28,1])
    # layer 1: conv + max_pool
    conv1 = conv2d(x, weights['wc1'], biases['bc1'], 1)
    conv1 = maxpool2d(conv1, 2)
    # layer 2: conv + max_pool
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'], 1)
    conv2 = maxpool2d(conv2, 2)
    
    # layer 3: fully connected layer (conv2 layer column should match wd1 row)
    # also the output from conv2 is 4d we need it in 2d for fully connected layer
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']),biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # dropout before final output layer
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # final output layer
    out = tf.add(tf.matmul(fc1, weights['out']),biases['out'])
    return out

In [26]:
weights = {
    # 5*5 convolution and 1 input 32 output channels
    'wc1': tf.Variable(tf.random_normal([5,5,1,32])),
    # 5*5 convolution and 32 input 64 output channels
    'wc2': tf.Variable(tf.random_normal([5,5,32,64])),
    # fully connected 7*7*64 input 512 output channels
    'wd1': tf.Variable(tf.random_normal([7*7*64,512])),
    # fully connected 512 input num_classes output channels
    'out': tf.Variable(tf.random_normal([512,num_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([512])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [34]:
# creating the model
logits = conv_net_2d(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)
# loss and optimization
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [35]:
with tf.Session() as sess:
    sess.run(init)
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})
        if step % display_step == 0 or step == 1:
            # only values for placeholders are needed not variables which are already assigned
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y, keep_prob:dropout})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + "{:.3f}".format(acc))
    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: mnist.test.images[:256], Y: mnist.test.labels[:256], keep_prob: 1.0}))

Step 1, Minibatch Loss= 59029.6016, Training Accuracy= 0.109
Step 10, Minibatch Loss= 35084.7422, Training Accuracy= 0.164
Step 20, Minibatch Loss= 19891.8203, Training Accuracy= 0.422
Step 30, Minibatch Loss= 12768.0820, Training Accuracy= 0.469
Step 40, Minibatch Loss= 13014.6895, Training Accuracy= 0.469
Step 50, Minibatch Loss= 8440.1973, Training Accuracy= 0.539
Step 60, Minibatch Loss= 8236.7744, Training Accuracy= 0.648
Step 70, Minibatch Loss= 7051.7437, Training Accuracy= 0.586
Step 80, Minibatch Loss= 4410.2573, Training Accuracy= 0.703
Step 90, Minibatch Loss= 5419.0713, Training Accuracy= 0.695
Step 100, Minibatch Loss= 3985.4753, Training Accuracy= 0.750
Step 110, Minibatch Loss= 2968.9658, Training Accuracy= 0.789
Step 120, Minibatch Loss= 2122.8726, Training Accuracy= 0.789
Step 130, Minibatch Loss= 3603.9929, Training Accuracy= 0.734
Step 140, Minibatch Loss= 3869.3489, Training Accuracy= 0.742
Step 150, Minibatch Loss= 1881.9287, Training Accuracy= 0.828
Step 160, Mini